In [ ]:
%pip install -qU pinecone langchain-pinecone langchain-google-genai langchain_community pypdf

# Accessing PineCone api key

In [2]:
from google.colab import userdata
import getpass
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(pinecone_api_key)

# Creating Index

In [3]:
index_name = "online-rag-project"

pc.create_index(
    name=index_name,
    dimension=768, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
index = pc.Index(index_name)

# Accessing gemini api key and embedding model

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
vector = embeddings.embed_query("This is the RAG project")

# store vectors

In [6]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# Loading pdf and spliting into chunks

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdf(pdf_path, chunk_size=1000, chunk_overlap=200):
    """Loads a PDF, splits it into chunks, and returns a list of Documents."""
    try:
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
    except FileNotFoundError:
        print(f"Error: File not found: {pdf_path}")
        return None
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return None

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    docs = text_splitter.split_documents(documents)
    return docs

# Example usage:
pdf_file = "/content/Generative AI Suggested Projects For Hackathon.pdf" # Make sure the PDF is in your Colab environment
loaded_docs = load_and_split_pdf(pdf_file)

if loaded_docs:
    print(f"Loaded {len(loaded_docs)} chunks from PDF.")
    # Now you can use loaded_docs with your vector store (Pinecone)
    # ... (Your existing code for indexing with Pinecone)
else:
    print("Failed to load or split the PDF.")

Loaded 6 chunks from PDF.


# embedd data and storing into pinecone vector database

In [8]:
vector_store.add_documents(loaded_docs)
print("PDF indexed successfully.")

PDF indexed successfully.


# Data Retrieval

In [ ]:
results = vector_store.similarity_search_with_score(
    "Machine Learning project"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [ ]:
results = vector_store.similarity_search(
    "Machine Learning project",
    k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [15]:
response = vector_store.similarity_search_with_relevance_scores(
    "Machine Learning project",
    k=2
)

# Iterate through the list of tuples (document, score)
for doc, score in response:
    print(doc.page_content)  # Access page_content from the Document object
    print(doc.metadata)
    print(score)

11.AI-PoweredPersonalFinanceAssistant
Description:Createatoolthathelpsusersmanagetheirfinancesbyanalyzingspendingpatternsandsuggestingsavingsstrategies.
Model:UtilizeNLPmodelsforfinancialdataanalysisandrecommendations.
12.AI-BasedArtGenerator
Description:Buildanapplicationthatgeneratesuniquepiecesofartorimagesbasedontextualdescriptions.
Model:UseDALL-EminiorCLIP-basedmodelsfortext-to-imagegeneration.
13.RecipeGenerator
Description:Createanappthatgeneratescookingrecipesbasedonavailableingredientsinputbytheuser.
Model:UtilizeGPT-3orsimilarmodelsfine-tunedforrecipegeneration.
14.AI-DrivenStoryGenerator
Description:Developanapplicationthatgeneratescreativeshortstoriesbasedonagivenprompt.
Model:UseGPT-3orotherstorygenerationmodelsavailableonHuggingFace.
15.Voice-BasedVirtualAssistant
Description:Createa virtualassistantthatrespondsto voicecommandstoperformtaskslikesettingremindersorsearchingtheweb.
{'page': 2.0, 'page_label': '3', 'source': '/content/Generative AI Suggested Projects For Hac